In [1]:
# Visualize prelim_CI_data.xlsx
import pandas as pd
import numpy as np
import altair as alt
import os

/home/sphsc/cse583/.blap/lib/python3.12/site-packages/numpy/_core/getlimits.py:552: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [44]:
pd.set_option('display.max_rows', 50)  # Show up to 1000 rows

In [37]:
subjids = ['deadFish', 'hydrolagusColliei_4', 'hydrolagusColliei_5', 'hydrolagusColliei_6', 'hydrolagusColliei_7', 'hydrolagusColliei_8', 'hydrolagusColliei_9']

# CI dataset
df_CIs = []
for subjid_idx, subjid in enumerate(subjids):
    pattern = f"/home/sphsc/cse583/fish_eeg/analysis/CI_csv/{subjid}/"
    all_files = os.listdir(pattern)
    for file_idx, cur_file in enumerate(all_files):
        tmp = pd.read_csv(os.path.join(pattern, cur_file))
        tmp['subjid'] = subjid
        tmp['test_id'] = cur_file.split('CI_')[1].replace('.csv', '')
        df_CIs.append(tmp)
df_CIs = pd.concat(df_CIs, ignore_index=True)


In [36]:
cur_file

'hydrolagusColliei_9_55Hz_125dB_CI_0.csv'

In [38]:
df_CIs

,subjid,freq,amp,lower_CI,upper_CI,test_id
0,deadFish,220,130,-0.126125,0.020731,0
1,deadFish,330,125,-0.031146,0.131323,0
2,deadFish,55,130,-0.086109,0.075130,0
3,deadFish,55,150,0.673900,0.864099,0
4,hydrolagusColliei_4,100,105,-0.138346,0.072922,0
...,...,...,...,...,...,...
196,hydrolagusColliei_9,50,145,7.408169,7.744027,0
197,hydrolagusColliei_9,55,110,-0.074493,0.112091,0
198,hydrolagusColliei_9,55,115,-0.128971,0.094606,0
199,hydrolagusColliei_9,55,120,0.256045,0.648667,0


In [56]:
# Manually recreate the dataframe from the values
data_dict = {
    'Subject_ID': df_CIs['subjid'].tolist(),
    'Test_ID': df_CIs['test_id'].tolist(),
    'Frequency': df_CIs['freq'].tolist(), #.tolist() removes index information, convert to pure python list, 
    'Amplitude': df_CIs['amp'].tolist(),
    'CI_lower': df_CIs['lower_CI'].tolist(),
    'CI_upper': df_CIs['upper_CI'].tolist()
}

df_new = pd.DataFrame(data_dict)
df_new['Midpoint'] = (df_new['CI_lower'] + df_new['CI_upper']) / 2
df_new['group'] = df_new['Subject_ID'] + '_' + df_new['Test_ID'].astype(str)
df_new = df_new[~df_new['Frequency'].isin([50, 880]) & (df_new['Amplitude'] != 145) & (df_new['Amplitude'] != 150)]
df_new

,Subject_ID,Test_ID,Frequency,Amplitude,CI_lower,CI_upper,Midpoint,group
0,deadFish,0,220,130,-0.126125,0.020731,-0.052697,deadFish_0
1,deadFish,0,330,125,-0.031146,0.131323,0.050088,deadFish_0
2,deadFish,0,55,130,-0.086109,0.075130,-0.005490,deadFish_0
4,hydrolagusColliei_4,0,100,105,-0.138346,0.072922,-0.032712,hydrolagusColliei_4_0
5,hydrolagusColliei_4,0,100,110,-0.200873,0.237800,0.018463,hydrolagusColliei_4_0
...,...,...,...,...,...,...,...,...
194,hydrolagusColliei_9,0,440,140,-0.122621,0.026986,-0.047818,hydrolagusColliei_9_0
197,hydrolagusColliei_9,0,55,110,-0.074493,0.112091,0.018799,hydrolagusColliei_9_0
198,hydrolagusColliei_9,0,55,115,-0.128971,0.094606,-0.017182,hydrolagusColliei_9_0
199,hydrolagusColliei_9,0,55,120,0.256045,0.648667,0.452356,hydrolagusColliei_9_0


In [62]:
# Add a zero column to your data
df_new['zero'] = 0

# Create area layer
area = alt.Chart(df_new).mark_area(opacity=0.3).encode(
    x=alt.X('Amplitude:Q', title='Stimulus Amplitude'),
    y=alt.Y('CI_lower:Q', title='95% Confidence Interval'),
    y2='CI_upper:Q',
    color=alt.Color('Frequency:O',title = 'Frequency (Hz)', scale=alt.Scale(scheme='viridis')),
    detail='group:N'  # Add this line
)

# Create y=0 line
zero_line = alt.Chart(df_new).mark_rule(color='black', strokeWidth=.5, strokeDash=[5, 5], opacity=0.3).encode(
    y=alt.datum(0)
)

# Create midpoint line
midpoint_line = alt.Chart(df_new).mark_line(strokeWidth=2).encode(
    x='Amplitude:Q',
    y='Midpoint:Q',
    color=alt.Color('Frequency:O', scale=alt.Scale(scheme='viridis')),
    detail='group:N'  # Add this line
)

# Create circles
circles = alt.Chart(df_new).mark_circle(size=60).encode(
    x='Amplitude:Q',
    y='Midpoint:Q',
    color=alt.Color('Frequency:O', scale=alt.Scale(scheme='viridis')),
    tooltip = ['Amplitude','Frequency','CI_lower','CI_upper'],
    detail='group:N'  # Add this line
)

# Layer then facet
chart = (area + zero_line + midpoint_line + circles).properties(
    width=300,
    height=300
).facet(
    column=alt.Column('Frequency:N', title='Frequency (Hz)'),
    columns = 5
).configure_axis(
    labelFontSize=13,
    titleFontSize=14
).configure_header(
    labelFontSize=18,
    titleFontSize=16  
).configure_legend(
    labelFontSize=13,
    titleFontSize=16
).interactive()

chart

alt.FacetChart(...)

In [48]:
# Create area layer
area = alt.Chart(df_new).mark_area(opacity=0.3).encode(
    x=alt.X('Amplitude:Q', title='Stimulus Amplitude'),
    y=alt.Y('CI_lower:Q', title='95% Confidence Interval', scale=alt.Scale(domain=[-0.5, 2.5])),
    y2='CI_upper:Q',
    color=alt.Color('Frequency:O', scale=alt.Scale(scheme='viridis'))
)

# Create horizontal line using mark_rule without x encoding
line = alt.Chart(df_new).mark_rule(color='black', strokeWidth=0.5, strokeDash=[5,5], opacity=0.5).encode(
    y=alt.datum(0)
)

# Layer then facet
chart = (area + line).properties(
    width=300,
    height=300
).facet(
    column='Frequency:N'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_header(
    labelFontSize=16,
    titleFontSize=16  
).configure_legend(
    labelFontSize=13,
    titleFontSize=16
)

chart

alt.FacetChart(...)